In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pandas
import urllib
import re

from selenium import webdriver
from time import sleep
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import NoSuchElementException, WebDriverException

import csv
import time

from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
#A quick look at https://www.leafly.com/explore/page-61/sort-alpha shows 
#that this is the last page of strains
pages = 61 
websites = []
strain_names = []

for i in range(pages+1):
    page_url = "https://www.leafly.com/explore/page-"+str(i)+"/sort-alpha"
    html = requests.get(page_url).text
    soup = BeautifulSoup(html,'html.parser')
    urls = soup.findAll("a",{"class":"ga_Explore_Strain_Tile"})
        
    for url in urls:
        strain_name = url.get('href')
        strain_names.append(strain_name)
        website = ("https://www.leafly.com" + strain_name)
        websites.append(website)

In [ ]:
#create url df
url_df = pd.DataFrame(websites, columns = ['url'])

In [ ]:
#create url csv
url_df.to_csv('urls.csv')

In [ ]:
#create strain_names csv
strains_df = pd.DataFrame(strain_names, columns = ['strain_name'])

In [ ]:
#create strain_names csv
strains_df.to_csv('strains.csv')

In [ ]:
#parse strain_name:
parsed =[]
for words in strains_df['strain_name']:
    parse = re.findall(r"[\w']+", words)
    parsed.append(parse)

In [ ]:
#create name and strain df

strain_type_df = pd.DataFrame(parsed, columns = ['strain', 'name1', 'name2',
                                    'name3', 'name4', 'name5', 'name6'])

In [ ]:
#drop last 5 columns - just fillers
strain_type_df.drop(df.tail(5).index,inplace=True)

In [ ]:
#merge name columns
strain_type_df['name'] = strain_type_df[strain_type_df.columns[1:]].apply(
    lambda x: ' '.join(x.dropna().astype(str)),
    axis=1)

In [ ]:
#drop unnecessary columns
strain_type_df.drop(['name1', 'name2', 'name3', 'name4', 'name5', 'name6'], axis=1, inplace=True)

In [ ]:
strain_type_df.to_csv('strain_type.csv')

In [ ]:
#number of duplicates
df['name'].duplicated().sum()

In [ ]:
url_df['url'].duplicated().sum()

In [ ]:
url_df.drop_duplicates(inplace=True)

In [ ]:
url_df.reset_index()

In [ ]:
len(url_df)

In [ ]:
#breaking down scraping into smaller buckets

urls_to_200 = urls_df['url'][:200]          #soups
urls_to_700 = urls_df['url'][200:700]       #soups_to_700
urls_to_1200 = urls_df['url'][700:1200]     #soups_to_1200
urls_to_1700 = urls_df['url'][1200:1700]    #soups_to_1700
urls_to_2200 = urls_df['url'][1700:2200]    #soups_to_2200
urls_to_2700 = urls_df['url'][2200:2700]    #soups_to_2700
urls_to_end = urls_df['url'][2700:]         #soups_to_end

#scraping all at once
urls_total = urls_df['url']                 #soups_total

In [ ]:
#scrape individual strain pages

soups_total = []

for url in urls_total:
    html = requests.get(url).text
    soup = BeautifulSoup(html,'html.parser')
    soups_total.append(soup)
    time.sleep(1)

In [ ]:
# with open("soups_total.html", "w") as file:
#     file.write(str(soups_total))

### Get Features From Tags

In [ ]:
#use this one

#collect positive effects

effect = []
filename1 = "effect.csv"

for i in tqdm(range(0, len(soups_total))): 
    names = []
    cont = soups_total[i].find("a",{"class":"active"})  
    names.append(cont)
    try:
        for name in names:
            strain_name = name.get('href') 
            strain = strain_name.split('/')
        div = soups_total[i].find('div', {'id': 'effects-tab-content'})
        first_child = div.findChildren('div', {'class': "histogram-label"})
        try:
            if len(first_child) == 5:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                    first_child[2].text, first_child[3].text, 
                        first_child[4].text]
                par = {key:values}
            
            elif len(first_child) == 4:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                          first_child[2].text, first_child[3].text]
                par = {key:values}
            
            elif len(first_child) == 3:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                          first_child[2].text]
                par = {key:values}
            
            elif len(first_child) == 2:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text]
                par = {key:values}
            
            elif len(first_child) == 1:
                key = strain[2]
                values = [first_child[0].text]
                par = {key:values}
            
            else:
                key = strain[2]
                values = None
                par = {key:values}       
        except:
            key = strain[2]
            values = None
            par = {key:values}
    except:
        key = strain[2]
        values = None
        par = {key:values}
    
        
    effect.append(par)

df_effect = pd.DataFrame(effect)
df_effect.to_csv(filename1)

#collect medical attributes

med = []
filename2 = "med.csv"

for i in tqdm(range(0, len(soups_total))): 
    names = []
    cont = soups_total[i].find("a",{"class":"active"})  
    names.append(cont)
    try:
        for name in names:
            strain_name = name.get('href') 
            strain = strain_name.split('/')
        div = soups_total[i].find('div', {'id': 'medical-tab-content'})
        first_child = div.findChildren('div', {'class': "histogram-label"})
        try:
            if len(first_child) == 5:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                    first_child[2].text, first_child[3].text, 
                        first_child[4].text]
                par = {key:values}
            
            elif len(first_child) == 4:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                          first_child[2].text, first_child[3].text]
                par = {key:values}
            
            elif len(first_child) == 3:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                          first_child[2].text]
                par = {key:values}
            
            elif len(first_child) == 2:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text]
                par = {key:values}
            
            elif len(first_child) == 1:
                key = strain[2]
                values = [first_child[0].text]
                par = {key:values}
            
            else:
                key = strain[2]
                values = None
                par = {key:values}       
        except:
            key = strain[2]
            values = None
            par = {key:values}
    except:
        key = strain[2]
        values = None
        par = {key:values}
    
        
    med.append(par)

df_med = pd.DataFrame(med)
df_med.to_csv(filename2)

#collect negative attributes

neg = []
filename3 = "neg.csv"

for i in tqdm(range(0, len(soups_total))): 
    names = []
    cont = soups_total[i].find("a",{"class":"active"})  
    names.append(cont)
    try:
        for name in names:
            strain_name = name.get('href') 
            strain = strain_name.split('/')
        div = soups_total[i].find('div', {'id': 'negatives-tab-content'})
        first_child = div.findChildren('div', {'class': "histogram-label"})
        try:
            if len(first_child) == 5:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                    first_child[2].text, first_child[3].text, 
                        first_child[4].text]
                par = {key:values}
            
            elif len(first_child) == 4:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                          first_child[2].text, first_child[3].text]
                par = {key:values}
            
            elif len(first_child) == 3:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                          first_child[2].text]
                par = {key:values}
            
            elif len(first_child) == 2:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text]
                par = {key:values}
            
            elif len(first_child) == 1:
                key = strain[2]
                values = [first_child[0].text]
                par = {key:values}
            
            else:
                key = strain[2]
                values = None
                par = {key:values}       
        except:
            key = strain[2]
            values = None
            par = {key:values}
    except:
        key = strain[2]
        values = None
        par = {key:values}
    
        
    neg.append(par)

df_neg = pd.DataFrame(neg)
df_neg.to_csv(filename3)

#collect flavors

flavors = []
filename4 = "flavors.csv"

for i in tqdm(range(0, len(soups_total))): 
    names = []
    cont = soups_total[i].find("a",{"class":"active"})  
    names.append(cont)
    try:
        for name in names:
            strain_name = name.get('href')
            strain = strain_name.split('/')
        first_child = soups_total[i].findAll('div',attrs={"class" : "flavor-name"})
        try:
            if len(first_child) == 3:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text, 
                        first_child[2].text]
                par = {key:values}
            elif len(first_child) == 2:
                key = strain[2]
                values = [first_child[0].text, first_child[1].text]
                par = {key:values}
            elif len(first_child) == 1:
                key = strain[2]
                values = [first_child[0].text]
                par = {key:values}
            else:
                key = strain[2]
                values = None
                par = {key:values}       
        except:
            key = strain[2]
            values = None
            par = {key:values}
    except:
        key = strain[2]
        values = None
        par = {key:values}
    
        
    flavors.append(par)

df_flavors = pd.DataFrame(flavors)
df_flavors.to_csv(filename4)  

# #collect parents of strains

# parents = []

# filename5 = 'parents.csv'

# for i in tqdm(range(0, len(soups_total))): 
#     children = soups_total[i].findAll('div',attrs={"class" : "strain-tile-footer"})
#     try:
#         if len(children) == 3:
#             key = children[0].text
#             values = [children[1].text, children[2].text]
#             par = {key:values}
#         elif len(children) == 2:
#             key = children[0].text
#             values = children[1].text
#             par = {key:values}        
#         else:
#             key = children[0].text
#             values = None
#             par = {key:values}
#     except:
#         None
#         #print("can't find that page")
    
#     parents.append(par)
   

# df_parents = pd.DataFrame(parents)
# df_parents.to_csv(filename5)

### Create Tokens

In [311]:
category_tokens = ['hyrid', 'indica', 'sativa']

flavor_tokens = ['1. ammonia', '2. ammonia', '3. ammonia',
                 '1. apple', '2. apple', '3. apple', 
                 '1. apricot', '2. apricot', '3. apricot',
                 '1. berry', '2. berry', '3. berry,
                 '1. blue cheese', '2. blue cheese', '3. blue cheese', 
                 '1. blueberry', '2. blueberry', '3. blueberry',
                 '1. butter', '2. butter', '3. butter', 
                 '1. cheese', '2. cheese', '3. cheese', 
                 '1. chemical', '2. chemical', '3. chemical', 
                 '1. chestnut', '2. chestnut', '3. chestnut', 
                 '1. citrus', '2. citrus', '3. citrus',
                 '1. coffee', '2. coffee', '3. coffee', 
                 '1. diesel', '2. diesel', '3. diesel', 
                 '1. earthy', '2. earthy', '3. earthy', 
                 '1. flowery', '2. flowery', '3. flowery',
                 '1. grape', '2. grape', '3. grape', 
                 '1. grapefruit', '2. grapefruit', '3. grapefruit', 
                 '1. honey', '2. honey', '3. honey',
                 '1. lavender', '2. lavender', '3. lavender',
                 '1. lemon', '2. lemon', '3. lemon', 
                 '1. lime', '2. lime', '3. lime', 
                 '1. mango', '2. mango', '3. mango', 
                 '1. menthol', '2. menthol', '3. menthol', 
                 '1. mint', '2. mint', '3. mint',
                 '1. nutty', '2. nutty', '3. nutty', 
                 '1. orange', '2. orange', '3. orange', 
                 '1. peach', '2. peach', '3. peach', 
                 '1. pear', '2. pear', '3. pear', 
                 '1. pepper', '2. pepper', '3. pepper',
                 '1. pine', '2. pine', '3. pine', 
                 '1. pineapple', '2. pineapple', '3. pineapple', 
                 '1. plum', '2. plum', '3. plum', 
                 '1. pungent', '2. pungent', '3. pungent', 
                 '1. rose', '2. rose', '3. rose', 
                 '1. sage', '2. sage', '3. sage', 
                 '1. skunk', '2. skunk', '3. skunk', 
                 '1. spicy/herbal', '2. spicy/herbal', '3. spicy/herbal', 
                 '1. strawberry', '2. strawberry', '3. strawberry',
                 '1. sweet', '2. sweet', '3. sweet', 
                 '1. tar', '2. tar', '3. tar', 
                 '1. tea', '2. tea', '3. tea', 
                 '1. tobacco', '2. tobacco', '3. tobacco', 
                 '1. tree fruit', '2. tree fruit', '3. tree fruit',
                 '1. tropical', '2. tropical', '3. tropical', 
                 '1. vanilla', '2. vanilla', '3. vanilla', 
                 '1. violet', '2. violet', '3. violet', 
                 '1. woody', '2. woody', '3. woody']

effect_tokens = ['anxious', 'aroused', 'creative', 'energetic',
                 'euphoric', 'focused', 'giggly', 'happy', 
                 'hungry', 'relaxed', 'sleepy', 'talkative',
                 'tingly', 'uplifted']

medical_tokens = ['cramps', 'depression', 'eye Pressure', 'fatigue',
                  'headaches', 'inflammation', 'insomnia', 
                  'lack of appetite', 'muscle spasms', 'nausea',
                  'pain', 'seizures', 'spasticity', 'stress']

negative_tokens = ['dry mouth', 'dry eyes', 'anxious',
                  'dizzy', 'paranoid', 'headache']

disease_tokens = ['ADD/ADHD',
                  "Alzheimer's", 'Anorexia', 'Anxiety', 'Arthritis',
                  'Asthma', 'Bipolar Disorder', 'Cachexia', 'Cancer',
                  "Crohn's Disease", 'Epilepsy', 'Fibromyalgia', 
                  'Gastrointestinal Disorder', 'Glaucoma', 'HIV/AIDS',
                  'Hypertension', 'Migraines', 'Multiple Sclerosis',
                  'Muscular Dystrophy', 'PMS', 'PTSD', "Parkinson's",
                  'Phantom Limb Pain', 'Spinal Cord Injury',
                  'Tinnitus', "Tourette's Syndrome"]

46

In [ ]:
with open('soups_total.html', 'r') as f:
    html = f.read()
soups_total = BeautifulSoup(html,'html.parser')

### Import CSVs

In [3]:
#import categories - hybrid, sativa, indica
cat_df = pd.read_csv('strain_category.csv')

#import flavors
flav_df = pd.read_csv('flavors.csv')

#import parents
par_df = pd.read_csv('parents.csv')

#import effects
eff_df = pd.read_csv('effect.csv')

#import medical
med_df = pd.read_csv('med.csv')

#import negative
neg_df = pd.read_csv('neg.csv')

### Pivot Tables/Table Formatting

In [4]:
#drop first column
def drop_column(df):
    df.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
drop_column(flav_df)
drop_column(par_df)
drop_column(eff_df)
drop_column(med_df)
drop_column(neg_df)
drop_column(cat_df)

In [6]:
#pivot tables so that column names are now indices
flav_series = flav_df.stack()
par_series = par_df.stack()
eff_series = eff_df.stack()
med_series = med_df.stack()
neg_series = neg_df.stack()

#convert from series to dataframe
flav_df = flav_series.to_frame(name='flavor')
par_df = par_series.to_frame(name='parent')
eff_df = eff_series.to_frame(name='effect')
med_df = med_series.to_frame(name='medical')
neg_df = neg_series.to_frame(name='negative')

#dataframe cleanup
def reset_index(df):
    df.reset_index(inplace=True)
    df.drop('level_0', axis=1, inplace=True)
    df.rename(columns={'level_1':'strain'}, inplace=True)

In [7]:
reset_index(flav_df)
reset_index(par_df)
reset_index(eff_df)
reset_index(med_df)
reset_index(neg_df)

In [8]:
#cat_df.reset_index('name', inplace=True)
cat_df.rename(columns={'strain':'category'}, inplace=True)
cat_df.rename(columns={'name':'strain'}, inplace=True)

#remove $ from parents table
par_df['strain'] = [x.strip('$') for x in par_df.strain]

#make all entries lowercase
flav_df = flav_df.apply(lambda x: x.astype(str).str.lower())
par_df = par_df.apply(lambda x: x.astype(str).str.lower())
eff_df = eff_df.apply(lambda x: x.astype(str).str.lower())
med_df = med_df.apply(lambda x: x.astype(str).str.lower())
neg_df = neg_df.apply(lambda x: x.astype(str).str.lower())
cat_df = cat_df.apply(lambda x: x.astype(str).str.lower())


#replace - with space in all tables
flav_df['strain'] = flav_df['strain'].str.replace('-',' ')
par_df['strain'] = par_df['strain'].str.replace('-',' ')
eff_df['strain'] = eff_df['strain'].str.replace('-',' ')
med_df['strain'] = med_df['strain'].str.replace('-',' ')
neg_df['strain'] = neg_df['strain'].str.replace('-',' ')
cat_df['strain'] = cat_df['strain'].str.replace('-',' ')


# #make all data lowercase
# def lowercase(df):
#     df = df.apply(lambda x: x.astype(str).str.lower())
#     return
    
# #replace dash with space in all tables
# def replace_dash(df, column):
#     df[column] = df[column].str.replace('-',' ')
#     return

In [9]:
def remove_char(df, column):
    df[column] = df[column].str.replace('[',' ')
    df[column] = df[column].str.replace(']',' ')
    df[column] = df[column].str.replace("'",'')
    

def new_columns_5(df, column):    
    
    # new data frame with split value columns 
    # making separate first flavor from new data frame 
    new1 = df[column].str.split(',', n = 1, expand = True)   
    df[str(column) + '_1'] = new1[0] 
    df[str(column) + '_2'] = new1[1]


    new2 = df[str(column) + '_2'].str.split(',', n = 1, expand = True)  
    df[str(column) + '_2']= new2[0] 
    df[str(column) + '_3']= new2[1] 
  
    new3 = df[str(column) + '_3'].str.split(',', n = 1, expand = True)
    df[str(column) + '_3']= new3[0]
    df[str(column) + '_4']= new3[1]


    new4 = df[str(column) + '_4'].str.split(',', n = 1, expand = True)
    df[str(column) + '_4']= new4[0]
    df[str(column) + '_5']= new4[1]


    #dropping old name columns 
    df.drop(columns =[column], inplace = True) 
    
def new_columns_3(df, column):    
    
    # new data frame with split value columns 
    # making separate first flavor from new data frame 
    new1 = df[column].str.split(',', n = 1, expand = True)   
    df[str(column) + '_1'] = new1[0] 
    df[str(column) + '_2'] = new1[1]


    new2 = df[str(column) + '_2'].str.split(',', n = 1, expand = True)  
    df[str(column) + '_2']= new2[0] 
    df[str(column) + '_3']= new2[1] 
  
    new3 = df[str(column) + '_3'].str.split(',', n = 1, expand = True)
    df[str(column) + '_3']= new3[0]

    #dropping old name columns 
    df.drop(columns =[column], inplace = True) 
    
def new_columns_2(df, column):    
    
    # new data frame with split value columns 
    # making separate first flavor from new data frame 
    new1 = df[column].str.split(',', n = 1, expand = True)   
    df[str(column) + '_1'] = new1[0] 
    df[str(column) + '_2'] = new1[1]

    #dropping old name columns 
    df.drop(columns =[column], inplace = True)

In [10]:
remove_char(flav_df, 'flavor')
remove_char(par_df, 'parent')
remove_char(eff_df, 'effect')
remove_char(med_df, 'medical')
remove_char(neg_df, 'negative')
remove_char(cat_df, 'category')

In [11]:
new_columns_5(eff_df, 'effect')
new_columns_5(med_df, 'medical')
new_columns_5(neg_df, 'negative')

In [12]:
new_columns_3(flav_df, 'flavor')
new_columns_2(par_df, 'parent')

In [13]:
par_df['strain'] = par_df['strain'].str.replace("'",'')

In [14]:
par_df.head(200)

,strain,parent_1,parent_2
0,100 og,og kush,None
1,12 year og,og kush,None
2,13 dawgs,g13,chemdawg
3,24k gold,kosher kush,tangie
4,3 bears og,triangle kush,None
5,100 og,og kush,None
6,12 year og,og kush,None
7,13 dawgs,g13,chemdawg
8,24k gold,kosher kush,tangie
9,3 bears og,triangle kush,None


### Create Final Merged Dataframe

In [16]:
#create properly formatted csvs

flav_final_df = pd.DataFrame(flav_df)
flav_final_df.to_csv('flav_final_df.csv')

par_final_df = pd.DataFrame(par_df)
par_final_df.to_csv('par_final_df.csv')

eff_final_df = pd.DataFrame(eff_df)
eff_final_df.to_csv('eff_final_df.csv')

med_final_df = pd.DataFrame(med_df)
med_final_df.to_csv('med_final_df.csv')

neg_final_df = pd.DataFrame(neg_df)
neg_final_df.to_csv('neg_final_df.csv')

cat_final_df = pd.DataFrame(cat_df)
cat_final_df.to_csv('cat_final_df.csv')

In [17]:
#import final csvs

flav_df = pd.read_csv('flav_final_df.csv')
par_df = pd.read_csv('par_final_df.csv')
eff_df = pd.read_csv('eff_final_df.csv')
med_df = pd.read_csv('med_final_df.csv')
neg_df = pd.read_csv('neg_final_df.csv')
cat_df = pd.read_csv('cat_final_df.csv')

In [18]:
#create master dataframe
drop_column(flav_df)
drop_column(par_df)
drop_column(eff_df)
drop_column(med_df)
drop_column(neg_df)
drop_column(cat_df)

final_df = pd.merge(cat_df, eff_df, how='outer')
final_df.drop_duplicates(inplace=True)
final_df = pd.merge(final_df, med_df, how='outer')
final_df.drop_duplicates(inplace=True)
final_df = pd.merge(final_df, neg_df, how='outer')
final_df.drop_duplicates(inplace=True)
final_df = pd.merge(final_df, flav_df, how='outer')
final_df.drop_duplicates(inplace=True)
final_df = pd.merge(final_df, par_df, how='outer')
final_df.drop_duplicates(inplace=True)

final_df.to_csv('final_df.csv')

In [289]:
total_reviews = pd.read_csv('total_reviews.csv')

In [292]:
final_df = pd.merge(final_df, total_reviews, how='outer')

In [294]:
final_df.drop('Unnamed: 0', axis=1, inplace=True)

In [297]:
final_df.drop_duplicates(inplace=True)

In [303]:
final_df.to_csv('final_df.csv')

In [304]:
final_df.head(10)

,category,strain,effect_1,effect_2,effect_3,effect_4,effect_5,medical_1,medical_2,medical_3,medical_4,medical_5,negative_1,negative_2,negative_3,negative_4,negative_5,flavor_1,flavor_2,flavor_3,parent_1,parent_2,review_total
0,hybrid,100 og,happy,uplifted,relaxed,euphoric,sleepy,stress,pain,depression,insomnia,lack of appetite,dry mouth,dizzy,dry eyes,anxious,paranoid,1. earthy,2. diesel,3. pungent,og kush,NaN,508.0
2,hybrid,98 white widow,creative,relaxed,energetic,sleepy,giggly,depression,muscle spasms,fatigue,pain,NaN,dry mouth,anxious,dizzy,dry eyes,headache,1. diesel,2. flowery,3. grape,NaN,NaN,10.0
4,sativa,1024,uplifted,energetic,happy,relaxed,focused,pain,fatigue,depression,stress,inflammation,dry mouth,dry eyes,dizzy,paranoid,anxious,1. tropical,2. mint,3. blueberry,NaN,NaN,49.0
6,hybrid,12 year og,focused,creative,hungry,euphoric,aroused,depression,pain,stress,NaN,NaN,anxious,dizzy,dry eyes,dry mouth,headache,1. diesel,NaN,NaN,og kush,NaN,2.0
8,hybrid,13 dawgs,sleepy,euphoric,focused,energetic,relaxed,pain,depression,stress,inflammation,fatigue,anxious,dry eyes,headache,dry mouth,NaN,1. diesel,2. apricot,3. citrus,g13,chemdawg,11.0
10,hybrid,22,energetic,focused,happy,uplifted,creative,pain,depression,fatigue,stress,muscle spasms,headache,NaN,NaN,NaN,NaN,1. diesel,2. pine,3. skunk,NaN,NaN,9.0
12,hybrid,24k gold,happy,relaxed,euphoric,uplifted,creative,stress,depression,pain,fatigue,insomnia,dry mouth,dry eyes,dizzy,anxious,headache,1. orange,2. citrus,3. sweet,kosher kush,tangie,219.0
14,indica,3 bears og,relaxed,giggly,sleepy,tingly,energetic,pain,stress,lack of appetite,muscle spasms,insomnia,dry mouth,dry eyes,headache,anxious,NaN,1. diesel,2. sweet,3. butter,triangle kush,NaN,7.0
16,hybrid,3 kings,happy,relaxed,uplifted,euphoric,energetic,stress,pain,depression,headaches,eye pressure,dry mouth,dry eyes,dizzy,headache,paranoid,1. earthy,2. diesel,3. pine,NaN,NaN,324.0
17,indica,303 og,focused,relaxed,uplifted,euphoric,sleepy,pain,depression,insomnia,muscle spasms,headaches,dry mouth,dry eyes,dizzy,anxious,NaN,1. diesel,2. earthy,3. woody,pre-98 bubba kush,chemdawg,82.0


In [20]:
len(final_df)

3073

In [80]:
urls = pd.read_csv('urls.csv')

In [194]:
urls_list = urls['url']

In [253]:
# #scrape for review totals and parents

parents = []
reviews = []

for url in tqdm(urls_list):
    html = requests.get(url).text
    soup = BeautifulSoup(html,'html.parser')

    try:
        container = soup.find("a",{"class":"active"})  
        strain_name = container.get('href') 
        strain = strain_name.split('/')
    except:
        None
    
    try:
        parent = soup.findAll('div',attrs={"class" : "strain-tile-footer"})
        parents.append(parent)
    except:
        parents.append(strain[2], None)
    
    try:
        container = soup.find("a",{"class":"active"})  
        strain_name = container.get('href') 
        strain = strain_name.split('/')
        review_total = soup.find('a',attrs={"onclick" : "window.strainHelpers.trackEvent('User Interactions', 'Strain Details', 'View all Reviews')"}).text
        review = review_total[review_total.find("(")+1:review_total.find(")")]
        rev_name = [strain[2], review]
    except:
        rev_name = [strain[2], None]
    
    reviews.append(rev_name)

time.sleep(1)

TypeError: append() takes exactly one argument (2 given)

In [305]:
#scrape for parents

parents = []

for url in tqdm(urls_list):
    html = requests.get(url).text
    soup = BeautifulSoup(html,'html.parser')

    try:
        children = soups_total[i].findAll('div',attrs={"class" : "strain-tile-footer"})
        parents.append(children)
    
    except:
        None

time.sleep(1)        
        
    
#     try:
#         if len(children) == 7:
#             key = children[0].text
#             values = [children[1].text, children[2].text,
#                      children[3].text, children[4].text,
#                      children[5].text, children[6].text]
#             par = {key:values}
#         elif len(children) == 6:
#             key = children[0].text
#             values = [children[1].text, children[2].text,
#                      children[3].text, children[4].text,
#                      children[5].text]
#             par = {key:values}        
#         elif len(children) == 5:
#             key = children[0].text
#             values = [children[1].text, children[2].text,
#                      children[3].text, children[4].text]
#             par = {key:values}        
#         elif len(children) == 4:
#             key = children[0].text
#             values = [children[1].text, children[2].text,
#                      children[3].text]
#             par = {key:values}         
#         elif len(children) == 3:
#             key = children[0].text
#             values = [children[1].text, children[2].text]
#             par = {key:values}        
#         elif len(children) == 2:
#             key = children[0].text
#             values = [children[1].text]
#             par = {key:values}       
#         else:
#             key = children[0].text
#             values = None
#             par = {key:values}
    
#         parents.append(parent)                  
    
    
    
    

SSLError: HTTPSConnectionPool(host='www.leafly.com', port=443): Max retries exceeded with url: /hybrid/gelato-33 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')")))

In [205]:
review_totals = pd.DataFrame(reviews)
review_totals.to_csv('review_totals.csv')

In [277]:
parents_final = pd.DataFrame(parents)
parents_final.to_csv('parents_final.csv')

In [208]:
review_totals.rename(columns=({0:'strain', 1:'review_total'}), inplace=True)

In [278]:
review_totals.to_csv('review_totals.csv')
parents_final.to_csv('parents_final.csv')

In [279]:
review_totals = pd.read_csv('review_totals.csv')
parents_final = pd.read_csv('parents_final.csv')

In [257]:
parents_final = []

for i in tqdm(range(0, len(parents))): 
    try:
        if len(parents[i]) == 7:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text,
                     parents[i][3].text, parents[i][4].text,
                     parents[i][5], parents[i][6]]
            par = {key.text:values.text}
        elif len(parents[i]) == 6:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text,
                 parents[i][3].text, parents[i][4].text,
                 parents[i][5]]
            par = {key.text:values.text}
        elif len(parents[i]) == 5:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text,
                 parents[i][3].text, parents[i][4].text]
            par = {key.text:values.text}
        elif len(parents[i]) == 4:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text,
                 parents[i][3].text]
            par = {key.text:values.text}    
        elif len(parents[i]) == 3:
            key = parents[i][0].text
            values = [parents[i][1].text, parents[i][2].text]
            par = {key.text:values.text}
        elif len(parents[i]) == 2:
            key = parents[i][0].text
            values = parents[i][1].text
            par = {key.text:values.text}        
        else:
            key = parents[i][0].text
            values = None
            par = {key.text:values}
    
        parents_final.append(par)
    
    except:
        None

In [225]:
review_totals.head()

,strain,review_total
0,100-og,508
1,98-white-widow,10
2,1024,49
3,12-year-og,2
4,13-dawgs,11


In [ ]:
"<([A-Za-z][A-Za-z0-9]*)\\b[^>]*>(.*?)</\\1>"

In [265]:
import re
test = re.findall("<([A-Za-z][A-Za-z0-9]*)\\b[^>]*>(.*?)</\\1>", '<div class="strain-tile-footer">$100 OG</div>')

In [267]:
def split_string(string):
    return re.findall("<([A-Za-z][A-Za-z0-9]*)\\b[^>]*>(.*?)</\\1>", '<div class="strain-tile-footer">$100 OG</div>')

In [266]:
test

[('div', '$100 OG')]

In [275]:
parents_final.head()

AttributeError: 'list' object has no attribute 'head'

In [274]:
parents_final['strain'] = parents_final['strain'].apply(split_string)

TypeError: list indices must be integers or slices, not str

In [ ]:
review_totals.head()

In [306]:
final = pd.read_csv('final_df.csv')

In [308]:
unique = final["effect_1"][~final["effect_1"].isin(final["effect_2"])].drop_duplicates()

In [309]:
unique

433      creative 
2915        happy 
Name: effect_1, dtype: object